In [1]:
# ------------------------------------------------------------------------------ #
# @Author:        F. Paul Spitzner
# @Email:         paul.spitzner@ds.mpg.de
# @Created:       2023-08-04 11:59:06
# @Last Modified: 2023-08-04 11:59:15
# ------------------------------------------------------------------------------ #
# ------------------------------------------------------------------------------ #

%reload_ext autoreload
%autoreload 2
%reload_ext ipy_dict_hierarchy
%reload_ext watermark

import logging
logging.basicConfig(
    format="%(asctime)s | %(levelname)-8s | %(name)-s > %(funcName)-s > %(message)s",
    level=logging.WARNING,
)
log = logging.getLogger("notebook")
log.setLevel("DEBUG")

import re
import glob
import h5py
import sys
import numpy as np
import xarray as xr
import pandas as pd
import dask

from tqdm import tqdm

sys.path.append('../')

from ana import utility as utl

# specify the path as closely as possible, we search recursively through all subdirs
data_dir = "../../../gnode/experiment_analysis/dat/"
meta_df = utl.all_unit_metadata(data_dir, reload=False)
meta_df = utl.load_spikes(meta_df)
meta_df = utl.default_filter(meta_df, trim=True)
meta_df = utl.merge_blocks(meta_df)
meta_df.tail()

2023-08-09 16:55:22,719 | DEBUG    | ana.utility > all_unit_metadata > Found 58 hdf5 files in /Users/paul/para/2_Projects/information_timescales/repo/gnode/experiment_analysis/dat
Fetching metadata from sessions: 100%|██████████| 58/58 [00:00<00:00, 63.44it/s]
2023-08-09 16:55:23,665 | DEBUG    | ana.utility > load_spikes > Loading spikes for 12584 units, 31190 rows for pandas dataframe.
Loading spikes for sessions: 100%|██████████| 58/58 [00:36<00:00,  1.61it/s]
2023-08-09 16:55:59,682 | DEBUG    | ana.utility > default_filter > Default quality checks, valid rows before: 30912
2023-08-09 16:55:59,687 | DEBUG    | ana.utility > default_filter > After rate check: 30842
2023-08-09 16:55:59,692 | DEBUG    | ana.utility > default_filter > After zero-length check: 30842
2023-08-09 16:55:59,704 | DEBUG    | ana.utility > default_filter > After minmum-duration check: 17646
Merging blocks for units: 100%|██████████| 11999/11999 [00:16<00:00, 710.75it/s]
2023-08-09 16:56:16,900 | DEBUG    | ana

,unit_id,stimulus,session,block,ecephys_structure_acronym,invalid_spiketimes_check,recording_length,firing_rate,filepath,num_spikes,spiketimes
23288,951190716,natural_movie_one_more_repeats,847657808,merged_3.0_and_8.0,LP,SUCCESS,1067.306152,3.486347,/Users/paul/para/2_Projects/information_timesc...,3721,"[<xarray.DataArray ()>\narray(3.1584473, dtype..."
23289,951190722,natural_movie_one_more_repeats,847657808,merged_3.0_and_8.0,LP,SUCCESS,1077.341553,2.770709,/Users/paul/para/2_Projects/information_timesc...,2985,"[<xarray.DataArray ()>\narray(0.40185547, dtyp..."
23290,951190724,natural_movie_one_more_repeats,847657808,merged_3.0_and_8.0,LP,SUCCESS,1076.488525,2.043682,/Users/paul/para/2_Projects/information_timesc...,2200,"[<xarray.DataArray ()>\narray(1.0895996, dtype..."
23291,951190819,natural_movie_one_more_repeats,847657808,merged_3.0_and_8.0,LP,SUCCESS,1075.182861,2.104758,/Users/paul/para/2_Projects/information_timesc...,2263,"[<xarray.DataArray ()>\narray(1.2353516, dtype..."
23292,951190848,natural_movie_one_more_repeats,847657808,merged_3.0_and_8.0,VISrl,SUCCESS,1071.963867,0.806930,/Users/paul/para/2_Projects/information_timesc...,865,"[<xarray.DataArray ()>\narray(1.5703125, dtype..."


In [2]:
meta_df = utl.load_metrics(meta_df, data_dir)
meta_df

2023-08-09 16:56:17,736 | DEBUG    | ana.utility > load_metrics > Loaded columns ['unit_id', 'image_selectivity_ns'] from /Users/paul/para/2_Projects/information_timescales/repo/gnode/experiment_analysis/dat/brain_observatory_1.1_analysis_metrics.csv
2023-08-09 16:56:17,949 | DEBUG    | ana.utility > load_metrics > Loaded columns ['unit_id', 'pref_dir_dm'] from /Users/paul/para/2_Projects/information_timescales/repo/gnode/experiment_analysis/dat/functional_connectivity_analysis_metrics.csv
2023-08-09 16:56:17,951 | DEBUG    | ana.utility > load_metrics > Matched 0 rows from meta_df in /Users/paul/para/2_Projects/information_timescales/repo/gnode/experiment_analysis/dat/brain_observatory_1.1_analysis_metrics.csv
2023-08-09 16:56:18,041 | DEBUG    | ana.utility > load_metrics > Matched 23293 rows from meta_df in /Users/paul/para/2_Projects/information_timescales/repo/gnode/experiment_analysis/dat/functional_connectivity_analysis_metrics.csv
2023-08-09 16:56:21,447 | WARNING  | ana.utilit

,unit_id,stimulus,session,block,ecephys_structure_acronym,invalid_spiketimes_check,recording_length,firing_rate,filepath,num_spikes,spiketimes,image_selectivity_ns,pref_dir_dm
0,951013153,natural_movie_one_more_repeats,787025148,3.0,VISam,SUCCESS,900.668806,21.542880,/Users/paul/para/2_Projects/information_timesc...,19403,[[[<xarray.DataArray (spiketimes: 30719)>\narr...,NaN,45.0
1,951013143,natural_movie_one_more_repeats,787025148,3.0,VISam,SUCCESS,900.718039,11.652925,/Users/paul/para/2_Projects/information_timesc...,10496,[[[<xarray.DataArray (spiketimes: 30719)>\narr...,NaN,90.0
2,951013133,natural_movie_one_more_repeats,787025148,3.0,VISam,SUCCESS,894.867666,0.448111,/Users/paul/para/2_Projects/information_timesc...,401,[[[<xarray.DataArray (spiketimes: 30719)>\narr...,NaN,-45.0
3,951013202,natural_movie_one_more_repeats,787025148,3.0,VISam,SUCCESS,900.265072,2.384853,/Users/paul/para/2_Projects/information_timesc...,2147,[[[<xarray.DataArray (spiketimes: 30719)>\narr...,NaN,45.0
4,951013187,natural_movie_one_more_repeats,787025148,3.0,VISam,SUCCESS,896.074534,0.706414,/Users/paul/para/2_Projects/information_timesc...,633,[[[<xarray.DataArray (spiketimes: 30719)>\narr...,NaN,90.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23288,951190716,natural_movie_one_more_repeats,847657808,merged_3.0_and_8.0,LP,SUCCESS,1067.306152,3.486347,/Users/paul/para/2_Projects/information_timesc...,3721,"[<xarray.DataArray ()>\narray(3.1584473, dtype...",NaN,-45.0
23289,951190722,natural_movie_one_more_repeats,847657808,merged_3.0_and_8.0,LP,SUCCESS,1077.341553,2.770709,/Users/paul/para/2_Projects/information_timesc...,2985,"[<xarray.DataArray ()>\narray(0.40185547, dtyp...",NaN,0.0
23290,951190724,natural_movie_one_more_repeats,847657808,merged_3.0_and_8.0,LP,SUCCESS,1076.488525,2.043682,/Users/paul/para/2_Projects/information_timesc...,2200,"[<xarray.DataArray ()>\narray(1.0895996, dtype...",NaN,0.0
23291,951190819,natural_movie_one_more_repeats,847657808,merged_3.0_and_8.0,LP,SUCCESS,1075.182861,2.104758,/Users/paul/para/2_Projects/information_timesc...,2263,"[<xarray.DataArray ()>\narray(1.2353516, dtype...",NaN,-45.0
